<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/tensorflow-1-public/blob/main/C1/W4/assignment/C1W4_Assignment_traducido.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

In [2]:
from tensorflow.keras.preprocessing.image import load_img

base_dir = "./data/"
happy_dir = os.path.join(base_dir, "happy/")
sad_dir = os.path.join(base_dir, "sad/")

print("Sample happy image:")
plt.imshow(load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}"))
plt.show()

print("\nSample sad image:")
plt.imshow(load_img(f"{os.path.join(sad_dir, os.listdir(sad_dir)[0])}"))
plt.show()


Sample happy image:


FileNotFoundError: ignored

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

sample_image  = load_img(f"{os.path.join(happy_dir, os.listdir(happy_dir)[0])}")
sample_array = img_to_array(sample_image)

print(f"Each image has shape: {sample_array.shape}")
print(f"The maximum pixel value used is: {np.max(sample_array)}")

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') is not None and logs.get('accuracy') > 0.999:
            print("\nReached 99.9% accuracy so cancelling training!")
            self.model.stop_training = True

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def image_generator():
    train_datagen = ImageDataGenerator(rescale=1/255)
    # Especifica el método para cargar las imágenes desde un directorio y pasa los argumentos adecuados:
    # - directory: debe ser una ruta relativa al directorio que contiene los datos
    # - targe_size: establecerlo igual a la resolución de cada imagen (excluyendo la dimensión de color)
    # - batch_size: número de imágenes que el generador produce cuando se le pide un siguiente lote. Establézcalo en 10.
    # - class_mode: Cómo se representan las etiquetas. Debe ser uno de "binario", "categórico" o "disperso".
    # Escoge el que mejor se adapte aquí dado que las etiquetas van a ser etiquetas binarias 1D.
    train_generator = train_datagen.flow_from_directory(directory   = base_dir,
                                                        target_size = (150, 150),
                                                        batch_size  = 10,
                                                        class_mode  = 'binary')
    return train_generator

In [ ]:
# Guarda tu generador en una variable
gen = image_generator()

In [ ]:
from tensorflow.keras import optimizers, losses

def train_happy_sad_model(train_generator):
    callbacks = myCallback()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    model.compile(loss=losses.binary_crossentropy,
                  optimizer=optimizers.RMSprop(learning_rate=0.001),
                  metrics=['accuracy']) 
    history = model.fit(xm= train_generator,
                        epochsm= 20,
                        callbacks = [callbacks]) 
    return history

In [ ]:
hist = train_happy_sad_model(train_generator=gen)

In [ ]:
print(f"Your model reached the desired accuracy after {len(hist.epoch)} epochs")
if not "accuracy" in hist.model.metrics_names:
    print("Use 'accuracy' as metric when compiling your model.")
else:
    print("The metric was correctly defined.")